In [2]:
############################################
##########  Import Libraries  #############
import cv2
import numpy as np
import os

In [3]:
###################  Import Keras Related Functions  ############
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pickle
import time

In [4]:
########################################################
###########  Mount Google Derive ######################
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Preprocessing Step

In [5]:
########################################################3
#######  Some Variables for Data   #####################
trainingDataset = []
classNumber = 0
img_size = 100
path = "/content/drive/My Drive/Mask_dataset/Training"      #path of data used for training

In [6]:
trainingDataset.clear()

###  Read all images category wise  #######
for folder in (os.listdir(path)):
  print(classNumber)
  fp = os.path.join(path,folder)
  print(fp)
  for eachImage in os.listdir(fp): 
    imagePath = os.path.join(fp,eachImage)
    img = (cv2.imread(imagePath,cv2.IMREAD_GRAYSCALE))/255
    resized=cv2.resize(img,(img_size,img_size))
    trainingDataset.append([resized,classNumber])
  classNumber = classNumber + 1

0
/content/drive/My Drive/Mask_dataset/Training/with_mask
1
/content/drive/My Drive/Mask_dataset/Training/without_mask


In [7]:
X = []      # for storing image data/features
Y = []      # for storing label of images
img_size = 100
np.random.shuffle(trainingDataset)      # shuffle dataset to prevent overfitting
for features, label in trainingDataset:
    X.append(features)
    Y.append(label)

In [10]:
X = np.array(X).reshape(-1, img_size, img_size, 1)    # reshape in other form --> numpy array
Y_binary = to_categorical(Y)                          #categorical type array
print('X',X[0])
print(Y_binary)
print("Type",type(Y_binary))
print(len(Y_binary))

X [[[0.92125237]
  [0.91769825]
  [0.9144255 ]
  ...
  [0.42781757]
  [0.44720574]
  [0.43458569]]

 [[0.91311765]
  [0.91362412]
  [0.91605784]
  ...
  [0.42264413]
  [0.4096492 ]
  [0.39532804]]

 [[0.92529412]
  [0.92529412]
  [0.92395588]
  ...
  [0.41440197]
  [0.40701764]
  [0.44739707]]

 ...

 [[0.72283107]
  [0.65187207]
  [0.57426457]
  ...
  [0.71477491]
  [0.66378631]
  [0.66976262]]

 [[0.67024488]
  [0.76777126]
  [0.74556279]
  ...
  [0.74735713]
  [0.7514593 ]
  [0.75599979]]

 [[0.66390134]
  [0.61528229]
  [0.71307751]
  ...
  [0.68798933]
  [0.71356249]
  [0.73248011]]]
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Type <class 'numpy.ndarray'>
1376


In [11]:
#  Saving features and labels of Data
np.save('X',X)
np.save('Y_binary',Y_binary)

In [ ]:
#############################################
#  Load saved numpy arrays  ##################
X=np.load('X.npy')
Y_binary=np.load('Y_binary.npy')


# CNN Architecture

In [29]:
############  Sequential Model (CNN Architecture)  ############
model = Sequential()

# 1st CONV2D & MaxPooling Layer
model.add(Conv2D(200, (3, 3), input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.3))

# 2nd Conv2D & MaxPooling Layer
model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.3))

"""
model.add(Conv2D(50, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.3))
"""
# flatten data for feeding to feed forward NN
model.add(Flatten())
model.add(Dropout(0.4))       #Dropout
model.add(Dense(50))          # Dense layer of 50 neurons
model.add(Activation('relu'))  

#final Layer 
model.add(Dense(2))
model.add(Activation('softmax'))  #softmax activation fn for classification

In [30]:
# Configuration of Model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

In [31]:
# split training and test data
X,test_data,Y_binary,test_target=train_test_split(X,Y_binary,test_size=0.2)

In [32]:
# Train the model
model.fit(X, Y_binary,batch_size=32,
          epochs=30, validation_split = 0.1)
 

# Saving Model
model.save("Rehan2_Mask.model") 

Epoch 1/30
25/25 [==============================] - 58s 2s/step - loss: 0.7918 - accuracy: 0.5249 - val_loss: 0.6638 - val_accuracy: 0.5795
Epoch 2/30
25/25 [==============================] - 57s 2s/step - loss: 0.6187 - accuracy: 0.6593 - val_loss: 0.5177 - val_accuracy: 0.7386
Epoch 3/30
25/25 [==============================] - 57s 2s/step - loss: 0.4281 - accuracy: 0.8103 - val_loss: 0.5025 - val_accuracy: 0.8068
Epoch 4/30
25/25 [==============================] - 57s 2s/step - loss: 0.3062 - accuracy: 0.9006 - val_loss: 0.3577 - val_accuracy: 0.8864
Epoch 5/30
25/25 [==============================] - 56s 2s/step - loss: 0.1847 - accuracy: 0.9389 - val_loss: 0.3540 - val_accuracy: 0.8864
Epoch 6/30
25/25 [==============================] - 57s 2s/step - loss: 0.1395 - accuracy: 0.9542 - val_loss: 0.4544 - val_accuracy: 0.8750
Epoch 7/30
25/25 [==============================] - 56s 2s/step - loss: 0.1175 - accuracy: 0.9533 - val_loss: 0.3722 - val_accuracy: 0.8864
Epoch 8/30
25/25 [==

In [34]:
#Evaluate model on testing data
print(model.evaluate(test_data,test_target))

7/7 [==============================] - 4s 603ms/step - loss: 0.1526 - accuracy: 0.9727
[0.1526496559381485, 0.9727272987365723]


# Make Prediction

In [49]:

def prepare(filepath):
    img_size = 100 
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)/255  
    img_resize = cv2.resize(img, (img_size, img_size))
    #cv2.imshow('image',img)
    #print(img_resize.reshape(-1, img_size, img_size, 1))  
    return img_resize.reshape(-1, img_size, img_size, 1)

image_path="1802008_10.jpg"
#image_read=cv2.imread(image_path)
prediction = model.predict(prepare(image_path))

print((prediction))

CATEGORIES = ["With Mask", "Without Mask"]
pred_class = CATEGORIES[np.argmax(prediction)]
print(pred_class)
#cv2.imshow(pred_class,image_read)


cv2.waitKey(0)
cv2.destroyAllWindows()

[[8.7405424e-05 9.9991262e-01]]
Without Mask


In [16]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 4.3MB/s 
     |████████████████████████████████| 112kB 10.8MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [17]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np
import pandas as pd
import sys

# Storing Trained Model

In [33]:
tfjs.converters.save_keras_model(model, 'models')

/usr/local/lib/python3.7/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
